### Import Dataset

test set are 2019 tournament data 

#### current best log-loss: 0.555960806126461 


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics

In [12]:
dftrain = pd.read_csv("X_train.csv")
dfeval = pd.read_csv("X_test.csv")

y_train = pd.read_csv("y_train.csv")
y_eval = pd.read_csv("y_test.csv")

In [13]:
print(dftrain)

      Unnamed: 0  team1_score WLoc  num_ot  team1_seed  strongseed  \
0              0           52    N       0           6           6   
1              1           61    N       0          15           2   
2              2           66    N       0          16          16   
3              3           64    N       0           9           8   
4              4           65    N       0           7           2   
...          ...          ...  ...     ...         ...         ...   
1241        1241           84    N       0           5           4   
1242        1242           50    N       0           9           9   
1243        1243           69    N       0           2           2   
1244        1244           95    N       0           1           1   
1245        1245           69    N       0           3           3   

     team2_region team1_region  season         host  ...  team1_adjtempo  \
0               X            X    2018      detroit  ...         68.7000   
1      

In [46]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sead_diff                   1246 non-null   int64  
 1   team1_seed                  1246 non-null   int64  
 2   team1_adjoe                 1246 non-null   float64
 3   team2_adjoe                 1246 non-null   float64
 4   team1_adjde                 1246 non-null   float64
 5   team2_adjde                 1246 non-null   float64
 6   team1_blockpct              1246 non-null   float64
 7   team2_blockpct              1246 non-null   float64
 8   team1_ap_preseason          522 non-null    float64
 9   team2_ap_preseason          529 non-null    float64
 10  team1_pt_team_season_wins   1246 non-null   float64
 11  team2_pt_team_season_wins   1246 non-null   float64
 12  team1_pt_overall_s16        1246 non-null   float64
 13  team2_pt_overall_s16        1246 

In [14]:
features = ["sead_diff", "team1_seed","team1_adjoe","team2_adjoe","team1_adjde","team2_adjde","team1_blockpct","team2_blockpct","team1_ap_preseason",
           "team2_ap_preseason","team1_pt_team_season_wins","team2_pt_team_season_wins",
           "team1_pt_overall_s16","team2_pt_overall_s16","team1_pt_coach_season_wins","team2_pt_coach_season_wins",
           "team1_pt_school_ncaa","team2_pt_school_ncaa"]

In [15]:
dftrain = dftrain[features]

dfeval = dfeval[features]

In [16]:
y_train = y_train.drop(y_train.columns[0],axis=1)

In [17]:
y_eval = y_eval.drop(y_eval.columns[0],axis=1)

### LightGBM 

#### log-loss: 0.555960806126461

梯度提升决策树模型

在梯度提升中，每个弱学习器都在上一个学习器的错误上进行训练，并专注于解决上一个学习器未能解决的问题。每个学习器的输出结果都会被结合到最终的决策中，从而提高预测的准确性。

In [55]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

n_HP_points_to_test = 500

In [56]:
clf = lgb.LGBMClassifier(max_depth=-1, random_state=1,metric='binary_logloss', n_jobs=4, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='neg_log_loss',
    cv=5,
    refit=True,
    random_state=1,
    verbose=True)

In [57]:
gs.fit(dftrain, y_train.values.ravel())

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


RandomizedSearchCV(cv=5,
                   estimator=LGBMClassifier(metric='binary_logloss',
                                            n_estimators=5000, n_jobs=4,
                                            random_state=1),
                   n_iter=500,
                   param_distributions={'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001ABCF82FF40>,
                                        'min_child_samples': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001ABD31F66D0>,
                                        'min_child_weight': [1e-05, 0.001, 0.01,
                                                             0.1, 1, 10.0,
                                                             100.0, 1000.0,
                                                             10000.0],
                                        'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001ABD1CCC070>,
                                   

In [58]:
y_pred=gs.predict_proba(dfeval)[:,1]

In [59]:
print("log_loss is: ", metrics.log_loss(y_eval,y_pred))
print("roc_auc is: ", metrics.roc_auc_score(y_eval,y_pred))

log_loss is:  0.555960806126461
roc_auc is:  0.8004464285714286


### Random Forest
#### log-loss: 

In [71]:
from sklearn.ensemble import RandomForestClassifier 
maximum_depth = 100
minimum_depth = 1
param_grid = {'max_depth': list(range(minimum_depth, maximum_depth+1))}

# The reason we choose RandomizedSearchCV instead of GridSearchCV because it's faster
# The Random Forest Model
gs_rf = RandomizedSearchCV(
    RandomForestClassifier(criterion='entropy', random_state=1), 
    param_grid,cv=3, n_jobs=4)

### SVM Classifer 